In [5]:
include("../../src/Finite_Vol/pnjl_FV.jl")
include("../../src/Finite_Vol/Rep.jl")
using Plots 
using LaTeXStrings
using DataFrames, CSV
using QuadGK, LinearAlgebra

In [2]:

function parametrize_deformation(R, δ;para=2.0,scale=1.0)
    """
    δ: 变形参数 (0 ≤ δ < ∞)
    para: 调节变形幅度的参数
    scale: +1 压扁 -1 拉长
    - δ = 0: 球形 (a=b=c=R)
    - δ > 0 (且 scale=1.0): 扁平椭球(a=b > c)
    - δ > 0 (且 scale=-1.0): 拉长椭球(a=b < c)
    - 表面积单调递增
    """
    V = (4/3)*π*R^3
    
    # 基于 β₂ 的简化
    β₂ = tanh(δ)  # 保证 β₂ < 1
  #@  para = 1.8
    a = R * (1 + para * β₂)^(scale * 2/3)
    b = a
    c = V / ((4/3)*π*a*b)
    
    return a, b, c
end



parametrize_deformation (generic function with 1 method)

In [39]:
R = 30.0
e = 0.0
a, b, c = parametrize_deformation(R, e;para=3.0,scale=-1.0)
ints = get_nodes_el(128, a, b, c, modes="D")
Ts = 10.0
muB = 300.0
X0 = [-1.8, -1.8, -2.2, 0.01, 0.01]  # phi_u, phi_d, phi_s, Phi1, Phi2
X0 = Tmu(Ts/hc, muB/hc, X0, ints)
phi = X0[1:3]
Phi1 = X0[4]
Phi2 = X0[5]
P0 = -Omega(phi, Phi1, Phi2, Ts/hc, muB/hc, ints)

a,b,c=30.0,30.0,30.0
 IR=0.06003080029233736


20.62553682907629

In [38]:
path = "../../data/FV/Trho_Maxwell_el=0.0.dat"
df01 = CSV.read(path, DataFrame)
Tc = df01.T
muc = df01.mu_star * 3
outpath = "../../data/FV/firte_00.csv"
df_out = DataFrame(T=Tc, mu_star=muc)
CSV.write(outpath, df_out)  

"../../data/FV/firte_00.csv"

In [42]:
nearest_index(T::Float64, Ttab::AbstractVector{<:Real}) = begin
    # 最近邻（与 Fortran 的最近邻选择等价，避免过严相等判断）
    idx = findmin(abs.(T .- Ttab))[2]  # 使用[2]访问索引
    Int(idx)
end

nearest_index (generic function with 1 method)

In [44]:

# mu_BC = 391*3 MEV
Ts = 50.0:2.0:400.0   # 单位：MeV
    # ===== 两套典型初值（与 Fortran 一致的“相位外形”）=====
X_CONF   = [-1.9, -1.9, -2.2, 0.0038, 0.0038]
X_DECONF = [-0.5, -0.5, -1.8, 0.80,   0.80  ]
firstline_path = "../../data/FV/firte_00.csv"
mu_B = 950   # MeV
df_in = CSV.read(firstline_path, DataFrame)
T_1stt   = df_in.T
mu_1stt  = df_in.mu_star
Tmax_1st = maximum(T_1stt)
# 初始用禁闭样；进入一阶区再切去禁闭样（与 Fortran 相同）
X0 = copy(X_CONF)
P0 = 20.62553682907629
data = zeros(length(Ts), 5)
NewX = similar(X0)
k_1st = 0   # 进入一阶区计数（Fortran 的 k_1st）
sol = zeros(length(Ts), 6)
sol[:, 1] .= Ts
println("mu_B = $mu_B MeV")
for (i, T) in enumerate(Ts)
    println("T = $T")

    # -------- 一阶区判据与初值切换（Fortran 逻辑复刻） --------
    # 仅当 T 低于一阶线最高温度时，一阶线才存在
    if T < Tmax_1st
        n  = nearest_index(T, T_1stt)
        μ1 = mu_1stt[n]                      # 该温度对应的一阶线 mu_1st
        if mu_B >= μ1                         # 进入一阶区的高 μ 一侧
            k_1st += 1
            if k_1st == 1
                # 第一次进入：强制切换到去禁闭样初值（Fortran 中的 -0.5 / 0.8 组合）
                X0 .= X_DECONF
            else
                # 已在一阶区内：延拓（Fortran 用 f=7 中心点上一解；这里用上一步 NewX）
                # 无需改动，当前 X0 已在上一轮末尾被设置为 NewX（见下方）
            end
        end
    end
    # ---------------------------------------------------------

    # 牛顿迭代/求解器
    X0 = Tmu(T/hc, mu_B/hc, X0, ints)
    sol[i, 2:end] = X0
    # 微扰
    data[i, :] = Ther_Rep(1.001*X0, T/hc, mu_B/hc, ints, P0)

    # 下一步的延拓初值（Fortran 的 “k>1 then 用上步中心点解” 的 Julia 等价）

end

df = DataFrame(data, [:T, :mu_B, :P,:E,:cs])
df_sol = DataFrame(sol, [:T, :phiu, :phid, :phis, :Phi, :Phi_bar])
CSV.write("rep_mu=$(mu_B).csv", df)
CSV.write("rep_sol_mu=$(mu_B).csv", df_sol)
println("Done!")


mu_B = 950 MeV
T = 50.0
T = 52.0
T = 54.0
T = 56.0
T = 58.0
T = 60.0
T = 62.0
T = 64.0
T = 66.0
T = 68.0
T = 70.0
T = 72.0
T = 74.0
T = 76.0
T = 78.0
T = 80.0
T = 82.0
T = 84.0
T = 86.0
T = 88.0
T = 90.0
T = 92.0
T = 94.0
T = 96.0
T = 98.0
T = 100.0
T = 102.0
T = 104.0
T = 106.0
T = 108.0
T = 110.0
T = 112.0
T = 114.0
T = 116.0
T = 118.0
T = 120.0
T = 122.0
T = 124.0
T = 126.0
T = 128.0
T = 130.0
T = 132.0
T = 134.0
T = 136.0
T = 138.0
T = 140.0
T = 142.0
T = 144.0
T = 146.0
T = 148.0
T = 150.0
T = 152.0
T = 154.0
T = 156.0
T = 158.0
T = 160.0
T = 162.0
T = 164.0
T = 166.0
T = 168.0
T = 170.0
T = 172.0
T = 174.0
T = 176.0
T = 178.0
T = 180.0
T = 182.0
T = 184.0
T = 186.0
T = 188.0
T = 190.0
T = 192.0
T = 194.0
T = 196.0
T = 198.0
T = 200.0
T = 202.0
T = 204.0
T = 206.0
T = 208.0
T = 210.0
T = 212.0
T = 214.0
T = 216.0
T = 218.0
T = 220.0
T = 222.0
T = 224.0
T = 226.0
T = 228.0
T = 230.0
T = 232.0
T = 234.0
T = 236.0
T = 238.0
T = 240.0
T = 242.0
T = 244.0
T = 246.0
T = 248.0
T = 250.0


In [30]:
df = DataFrame(data, [:T, :mu_B, :P, :E, :cs2])
CSV.write("cs_e=$e.csv", df)

"cs_e=1.0.csv"

In [31]:
df2 = DataFrame(sol, [:mu_B, :T, :phi_u, :phi_d, :phi_s, :Phi1, :Phi2])
CSV.write("sol_e=$e.csv", df2)

"sol_e=1.0.csv"